# Automated DCF Valuation Model - For Apple Inc #

### 

In [4]:
import yfinance as yf 
import pandas as pd 


### The Fetcher ### 

# Creating ticker object 
company = yf.Ticker("AAPL")

# Getting recent financial statements
cash_flow = company.cash_flow
income_stmt = company.income_stmt
history = company.history(period ="1d")


# Selecting usefull rows from cash flow statement 
fcf_row = cash_flow.loc['Free Cash Flow']


#Selecting usefull rows from income statement
current_rev = income_stmt.loc['Total Revenue'].iloc[0]
previous_rev = income_stmt.loc['Total Revenue'].iloc[1]
diluted_avshares = income_stmt.loc['Diluted Average Shares'].iloc[0]
stock_price = history['Close'].iloc[0]

#Calculating Revenew Growth
rev_growth = (current_rev/previous_rev)-1 

### The Engine ###

# Setting up Model Drivers
discount_rate = 0.1
terminal_growth = 0.03
projection_yrs = 5

# Estimating DCF, FCF & PV over the next 5 years 
dcf_vals = []
running_fcf = fcf_row.iloc[0]
for year in range(1, projection_yrs +1):
    new_fcf = running_fcf * (1+rev_growth)
    running_fcf = new_fcf
    pv = running_fcf / (1 + discount_rate) ** year
    dcf_vals.append(pv)
   

# Summing PV's and Gathering Terminal Value 
terminal_value = (running_fcf * (1+terminal_growth)) / (discount_rate - terminal_growth)
discount_tv = terminal_value / (1+discount_rate)** projection_yrs

### The Output ###

# Calculating Target Price
total_value = sum(dcf_vals) + discount_tv
target_price = total_value / diluted_avshares
upside = ((target_price - stock_price) / stock_price) 
print(f"Estimated Price:${target_price:,.2f}",f"Current Price:${stock_price:,.2f}", f"Upside/Downside:{upside:,.2%}")


Estimated Price:$111.95 Current Price:$278.66 Upside/Downside:-59.83%
